# Scoping du projet

Le jeu de données que nous avons la charge de travailler contient des informations sous forme de paires de phrases (prémisse et hypothèse) avec des métadonnées telles que les identifiants, les abréviations de langue et les étiquettes de langue. Ce jeu de données semble être utilisé pour des tâches de compréhension de langage naturel, comme la classification d'inférence ou la prédiction de la relation entre la prémisse et l'hypothèse. En ce qui nous concerne, nous sommes charger de classifier la colonne "promised" + "hypothesis" en fonction du label correspondant.

1. **Objectif du projet** :
   - L'objectif principal est de développer un modèle de Deep Learning capable de prédire la relation entre la prémisse et l'hypothèse dans chaque paire de phrases.

2. **Tâche** :
   - Tâche de classification : Prédire la relation entre la prémisse et l'hypothèse. Les classes peuvent être les différentes relations possibles telles que "Contradiction", "Entailment" et "Neutral".

3. **Métriques d'évaluation** :
   - Les métriques d'évaluation appropriées pour cette tâche pourraient inclure la précision, le rappel, la F1-score, et la matrice de confusion.

4. **Collecte et préparation des données** :
   - Prétraitement : Nettoyer le texte en retirant les caractères spéciaux, en normalisant la casse, etc.
   - Diviser le jeu de données en ensembles d'entraînement, de validation et de test.

5. **Architecture du modèle** :
   - Choisissez une architecture appropriée pour la tâche, telle qu'un modèle basé sur Transformer (BERT, RoBERTa, etc.) ou des réseaux de neurones récurrents (LSTM, GRU) en fonction de la complexité de la tâche.

6. **Entraînement et évaluation du modèle** :
   - Entraînez le modèle sur l'ensemble d'entraînement et ajustez les hyperparamètres pour optimiser les performances.
   - Évaluez le modèle sur l'ensemble de validation pour surveiller le surapprentissage et ajuster les paramètres en conséquence.

7. **Tests finaux** :
   - Évaluez le modèle final sur l'ensemble de test pour obtenir une évaluation impartiale de ses performances.

8. **Communication des résultats** :
   - Présentez les performances du modèle à l'aide de métriques appropriées dans un rapport ou une présentation.

9. **Itérations** :
   - En fonction des performances du modèle, effectuez des itérations pour ajuster les hyperparamètres, essayer différentes architectures et améliorer les résultats.

10. **Gestion des ressources** :
   - Assurez-vous d'avoir suffisamment de puissance de calcul pour l'entraînement des modèles et de prévoir des ressources pour les expériences itératives.

11. **Limitations et risques** :
   - Identifiez les limitations potentielles du modèle, telles que la nécessité d'un grand volume de données annotées et les défis liés aux langues moins courantes.

# Data

In [15]:
#!pip install xlrd



In [16]:
import pandas as pd

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Charger le jeu de données
df = pd.read_excel('/content/drive/Othercomputers/Mon ordinateur/DIT MASTER1 2022/MASTER 2/DEEP LEARNING/devoir/contradictory-my-dear.xlsx')

In [19]:
df.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [20]:
len(df)

12120

In [21]:
# Compter le nombre de labels distincts
num_unique_labels = df['label'].nunique()
print(num_unique_labels)

3


In [22]:
# Afficher les labels distincts
unique_labels = df['label'].unique()
print(f"Labels distincts : {unique_labels}")

Labels distincts : [0 2 1]


In [23]:
# Compter le nombre de langues distinctes
num_unique_language = df['language'].nunique()
print(f"Nombre de langues distinctes : {num_unique_language}")

Nombre de langues distinctes : 15


In [24]:
# Afficher les langues distincts
unique_language = df['language'].unique()
print(f"langues distinctes : {unique_language}")


langues distinctes : ['English' 'French' 'Thai' 'Turkish' 'Urdu' 'Russian' 'Bulgarian' 'German'
 'Arabic' 'Chinese' 'Hindi' 'Swahili' 'Vietnamese' 'Spanish' 'Greek']


# Modeling

In [25]:
#!pip install transformers
#!pip install wandb

In [125]:
from transformers import AutoTokenizer, BertModel, AdamW, PreTrainedModel, BertForPreTraining
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import pandas as pd
import torch.nn.functional as F
from huggingface_hub import PyTorchModelHubMixin


from tqdm import tqdm

from sklearn.model_selection import train_test_split

import wandb

In [153]:
config = {
    "num_epochs" : 2,
    "max_length" : 80,
    "num_classes" : 3,
    "learning_rate" : 2e-5,
    "batch_size" : 16,
    "shuffle" : True,
    "model_name" : 'bert-base-multilingual-cased',
    "device" : torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    "file" : "/content/drive/Othercomputers/Mon ordinateur/DIT MASTER1 2022/MASTER 2/DEEP LEARNING/devoir/contradictory-my-dear.xlsx"
}

In [93]:
class MyDataset(Dataset):
    def __init__(self, excel_file, tokenizer, max_length):
        self.df = pd.read_excel(excel_file)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        item = self.df.iloc[index]
        premise = item['premise']
        hypothesis = item['hypothesis']
        label = item['label']

        inputs = self.tokenizer(premise, hypothesis, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')

        return {
            "input_ids" : inputs['input_ids'],
            "attention_mask" : inputs['attention_mask'],
            "label" : torch.tensor(label)
        }

In [94]:
# dataloader
def dataloader(dataset, batch_size, shuffle):
    return DataLoader(dataset=dataset, batch_size= batch_size, shuffle=shuffle)

In [132]:
class CustomModel(nn.Module, PyTorchModelHubMixin):
    def __init__(self, model_name, num_classes):
        super(CustomModel, self).__init__()
        self.pretrained_model = BertModel.from_pretrained(model_name) # 768 corresponds to the BERT's output size
        self.classifier = nn.Linear(768, num_classes)   # MLP

    def forward(self, input_ids, attention_mask):
        output = self.pretrained_model(input_ids=input_ids, attention_mask=attention_mask) # (bacth 768)
        output = self.classifier(output.last_hidden_state)
        return output

In [96]:
# training step
def train_step(model, train_loader, optimizer, loss_fn, device):
    model.train()

    total_loss = 0

    for data in tqdm(train_loader, total = len(train_loader)):
        inputs = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        targets = data['label'].to(device)

        optimizer.zero_grad()
        output = model(inputs.squeeze(1), attention_mask)
        loss = loss_fn(output, F.one_hot(targets, num_classes=3))

        # backward
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return  total_loss / len(train_loader)

In [97]:
def validation_step(model, validation_loader, loss_fn, device):
    total_loss = 0
    correct_prediction = 0

    with torch.no_grad():
        for data in tqdm(validation_loader, total = len(validation_loader)):
            input_ids = data['input_ids'].squeeze(1).to(device)
            attention_mask = data['attention_mask'].to(device)
            targets = data['label'].to(device)

            output = model(input_ids = input_ids, attention_mask = attention_mask)
            loss = loss_fn(output, F.one_hot(targets, num_classes=3))

            total_loss += loss.item()

            pred = torch.max(torch.softmax(output, dim=1), dim = 1)
            correct_prediction += torch.sum(pred.indices==F.one_hot(targets, num_classes=3))


        return  total_loss / len(validation_loader), 100*correct_prediction/len(validation_loader)

In [154]:
# main
def main():
    wandb.init('bert_classification')

    tokenizer = AutoTokenizer.from_pretrained(config['model_name'])
    # dataset
    dataset = MyDataset(excel_file=config['file'], tokenizer=tokenizer, max_length=config['max_length'])

    train_dataset, validation_dataset = train_test_split(dataset, test_size=0.2)

    # trainloader
    train_loader = dataloader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    validation_loader = dataloader(validation_dataset, batch_size=config['batch_size'], shuffle=False)

    # model
    model = CustomModel(model_name=config['model_name'], num_classes=config['num_classes'])
    model.to(config['device'])

    loss_fn = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=config['learning_rate'])

    for epoch in range(config['num_epochs']):
        loss_train = train_step(model, train_loader, optimizer, loss_fn, config['device'])
        loss_validatiton, accuracy = validation_step(model, validation_loader, loss_fn, config['device'])

        wandb.log({
            "loss_train":loss_train,
            "loss_validatiton":loss_validatiton,
            "accuracy":accuracy
            })
    # push model to hub
    model.push_to_hub("billfass/multilingual_bert_model_classiffication")
    tokenizer.push_to_hub("billfass/multilingual_bert_model_classiffication")
    model.pretrained_model.config.push_to_hub("billfass/multilingual_bert_model_classiffication")

In [ ]:
!git config --global credential.helper store
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your

In [155]:
main()

accuracy,▁▇██▇▇▇▇▇▇
loss_train,█▇▅▄▃▂▁▁▁▁
loss_validatiton,▂▁▁▂▃▄▇▆█▆
accuracy,3492.76318
loss_train,0.0547
loss_validatiton,0.87374


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 152/152 [00:03<00:00, 43.53it/s]


pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

#Deployment